# IMPORTS

In [36]:
from IPython.display import display
import time

import datetime
import importlib
import re
import multiprocessing
import subprocess

import scipy.stats
import random
import gzip
import glob
import io
import os
import sys

In [2]:
if not '/users/ldog/moyon/Thesis//scripts/' in sys.path:
    sys.path.insert(0,'/users/ldog/moyon/Thesis//scripts/')

import dataframes
from dataframes import * 
importlib.reload(dataframes)

<module 'dataframes' from '/users/ldog/moyon/Thesis//scripts/dataframes.py'>

In [239]:
import tempfile
import pybedtools as pbt
tmp_dir = "/localtmp/moyon/"
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
tmp_dir = tempfile.TemporaryDirectory(dir=tmp_dir).name
os.makedirs(tmp_dir)
print("Temporary bedfiles are in {}".format(tmp_dir))

pbt.set_tempdir(tmp_dir)

Temporary bedfiles are in /localtmp/moyon/tmppe7pa7bg


In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
husl = sns.color_palette('husl',32)
sns.palplot(husl)
plt.close()
plt.style.use('seaborn-notebook')

SMALL_SIZE=16
MEDIUM_SIZE=20
BIGGER_SIZE=22

mpl.rc('font',size=SMALL_SIZE)
mpl.rc('axes',titlesize=SMALL_SIZE)
mpl.rc('axes',labelsize=MEDIUM_SIZE)
mpl.rc('xtick',labelsize=SMALL_SIZE)
mpl.rc('ytick',labelsize=SMALL_SIZE)
mpl.rc('legend',fontsize=SMALL_SIZE)
mpl.rc('figure',titlesize=BIGGER_SIZE)

In [257]:
command_line_merge = """bedops --partition {inputfile} | awk -F "\\t" '{{ if ($2<$3) {{print $0}} }}' | bedtools sort -i - | bedmap --echo --echo-map-id --delim "\\t" --multidelim {delim} - {inputfile} > {outputfile}"""

In [256]:
print(command_line_merge)

bedops --partition {inputfile} | awk -F "\t" '{{ if ($2<$3) {{print $0}} }}' | bedtools sort -i - | bedmap --echo --echo-map-id --delim "	" --multidelim {delim} - {inputfile} > {outputfile}


In [342]:
def from_list_genes_to_kv_pairs(str_list_genes):
    """
    
    Transforms:
        'PLEKHN1|AGRN|AGRN|HES4|HES4'
    Into:
        'score:1,targets:PLEKHN1|score:2,targets:HES4;AGRN'
    """
    return '|'.join(pd.Series(str_list_genes.split('|')).value_counts().astype(str).reset_index().set_axis(['targets','score'],axis=1,inplace=False
                                                                                                     ).groupby('score')['targets'].apply(lambda v: ';'.join(v)
                                                                                                                                          ).reset_index().apply(lambda row: dataframes.convert_keyvalue_pairs_to_str(list(zip(row.index, row.values))),axis=1
                                                                                                                                          ).values)
                                                                                                    

# MAIN

For all resources, we need to assess that there's no overlap between two regions in the same resource ; otherwise we need to decide of a way to merge the information into a single one.

Then, the resources can be merged into a single file, where each unique region is associated to a set of target genes with format as follow :

```source:<namesource>;targets:<target1,target2>;score:<score>|source:<namesource>;targets:<target1,target2>;score:<score>```


From there I will derive a table with the information on shared genes.

## GENE NAMES

For unification, we need to make sure that we use all the same gene names across all files.

In [6]:
gencode_gene_table = pd.read_table("/users/ldog/moyon/Thesis/RegulationData/hg19/gene_tables/gencode_ensemblIDs_to_geneNames.tsv.gz")
# Here I add a set of mapping from the stripped names and ids.
#NOTE: not necessary anymore since I decided to remove the suffix from the gene IDs in the GENCODE files generation.
# stripped_ids_df = gencode_gene_table.copy()
# stripped_ids_df.gene_id = stripped_ids_df.gene_id.str.split('.', expand=True).iloc[:,0]
# gencode_gene_table = pd.concat([stripped_ids_df, gencode_gene_table]).reset_index(drop=True)


hgnc_gene_table = pd.read_table("/users/ldog/moyon/Thesis/RegulationData/hg19/gene_tables/20181122_HUGO_genenames_table.tsv")

print("GENCODE:\n\tNb of unique gene_id: {:,}\n\tNb of unique gene_name: {:,}".format(len(gencode_gene_table.gene_id.unique()),
                                                                                  len(gencode_gene_table.gene_name.unique())))
print("HGNC:\n\tNb of unique gene_id: {:,}\n\tNb of unique approved symbol: {:,}".format(len(hgnc_gene_table['Ensembl gene ID'].unique()),
                                                                                     len(hgnc_gene_table['Approved symbol'])))

display(gencode_gene_table.head())
display(hgnc_gene_table.head())

GENCODE:
	Nb of unique gene_id: 56,869
	Nb of unique gene_name: 56,706
HGNC:
	Nb of unique gene_id: 35,498
	Nb of unique approved symbol: 46,016


,gene_id,gene_name
0,ENSG00000223972,DDX11L1
1,ENSG00000227232,WASH7P
2,ENSG00000243485,MIR1302-2HG
3,ENSG00000237613,FAM138A
4,ENSG00000268020,OR4G4P


,HGNC ID,Approved symbol,Approved name,Status,Previous symbols,Synonyms,Chromosome,Accession numbers,RefSeq IDs,Previous name,Ensembl gene ID
0,HGNC:5,A1BG,alpha-1-B glycoprotein,Approved,NaN,NaN,19q13.43,NaN,NM_130786,NaN,ENSG00000121410
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,Approved,"NCRNA00181, A1BGAS, A1BG-AS",FLJ23569,19q13.43,BC040926,NR_015380,"non-protein coding RNA 181, ""A1BG antisense RN...",ENSG00000268895
2,HGNC:24086,A1CF,APOBEC1 complementation factor,Approved,NaN,"ACF, ASP, ACF64, ACF65, APOBEC1CF",10q11.23,AF271790,NM_014576,NaN,ENSG00000148584
3,HGNC:6,A1S9T~withdrawn,"symbol withdrawn, see UBA1",Symbol Withdrawn,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HGNC:7,A2M,alpha-2-macroglobulin,Approved,NaN,"FWP007, S863-7, CPAMD5",12p13.31,"BX647329, X68728, M11313",NM_000014,NaN,ENSG00000175899


In [7]:
MAP_TO_NAMEs = gencode_gene_table.set_index('gene_id')['gene_name'].to_dict()

## Enhancers

In [21]:
def get_updated_symbols(gene_list, hgnc_gene_table):
    """ from a list of gene symbols, get updated version from the Hugo gene table.
    
    gene symbols in gene_list should not be in the "Approved symbol" section of the table.
    Hence we look for them in the Previous symbols and Synonyms sections.
    """
    
    ignored_genes = []
    false_positives_prev = {}
    false_positives_syn = {}
    genes_not_found = []
    update_gene = {}
    
    for g in gene_list:
        if len(g) < 3 :
            ignored_genes.append(g)
            continue
            
        previous_symbols_candidates = hgnc_gene_table['Previous symbols'].str.contains(g).replace(np.nan,False)

        not_found=True

        if not previous_symbols_candidates.sum()==0:
            potential_candidates = hgnc_gene_table.loc[previous_symbols_candidates,:]
            # Now check if it is an exact match:
            all_previous_symbols = []
            for r, values in potential_candidates.loc[:,['Approved symbol','Previous symbols']].iterrows():
                previous_symbols = re.split(',| ',values['Previous symbols'])
                all_previous_symbols += previous_symbols
                if g in previous_symbols:
                    not_found = False
                    update_gene[g] = values['Approved symbol']
                    break

            else:
                false_positives_prev[g] = all_previous_symbols

        synonyms_candidates = hgnc_gene_table['Synonyms'].str.contains(g).replace(np.nan,False)

        if not synonyms_candidates.sum()==0:
            potential_candidates = hgnc_gene_table.loc[synonyms_candidates,:]
            # Now check if it is an exact match:
            all_synonyms = []
            for r, values in potential_candidates.loc[:,['Approved symbol','Synonyms']].iterrows():
                synonyms = re.split(',| ', values['Synonyms'])
                all_synonyms += synonyms
                if g in synonyms:
                    not_found = False
                    update_gene[g] = values['Approved symbol']
                    break
            else:
                false_positives_syn[g] = all_synonyms
        if not_found:
            genes_not_found.append(g)
            
    return update_gene, ignored_genes, false_positives_prev, false_positives_syn, genes_not_found

### DYOGEN

In [9]:
path_dyo = "/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/DYOGEN/raw/hg19_CNEs_PEGASUS.data.gz"
df_dyo = pd.read_table(path_dyo)
display(df_dyo.head())

,chromosome,start,end,ID,target,segregation.score,length,phyloP,danRer7.CNE.target.orthology,jumping,danRer7
0,chr1,25639,25727,chr1_1,ENSG00000187608_ENSG00000188976_ENSG0000026817...,0.01,89,2.077020,Nothing,NaN,danRer7.No
1,chr1,46734,46765,chr1_6,ENSG00000185097_ENSG00000235249_ENSG00000186092,0.43,32,-0.038677,Nothing,NaN,danRer7.No
2,chr1,46774,46841,chr1_7,ENSG00000186092,0.48,68,0.245910,Nothing,Flanking,danRer7.No
3,chr1,46842,46888,chr1_8,ENSG00000186092,0.48,47,0.192826,Nothing,Flanking,danRer7.No
4,chr1,47234,47255,chr1_9,ENSG00000185097_ENSG00000235249,0.31,22,0.414286,Nothing,NaN,danRer7.No


We reformat the dataset into a more readable format, unified with other resources. 

In [10]:
red_df_dyo = df_dyo.loc[:,['chromosome','start','end','segregation.score']].assign(targets=df_dyo.target.apply(lambda v: ','.join(v.split('_'))),
                                                                                   name='PEGASUS_'+df_dyo['ID']).rename(columns={'chromosome':'chrom','segregation.score':'score'}
                                                                                                                        ).loc[:,['chrom','start','end','name','score','targets']]
display(red_df_dyo.head())

,chrom,start,end,name,score,targets
0,chr1,25639,25727,PEGASUS_chr1_1,0.01,"ENSG00000187608,ENSG00000188976,ENSG0000026817..."
1,chr1,46734,46765,PEGASUS_chr1_6,0.43,"ENSG00000185097,ENSG00000235249,ENSG00000186092"
2,chr1,46774,46841,PEGASUS_chr1_7,0.48,ENSG00000186092
3,chr1,46842,46888,PEGASUS_chr1_8,0.48,ENSG00000186092
4,chr1,47234,47255,PEGASUS_chr1_9,0.31,"ENSG00000185097,ENSG00000235249"


##### Gene IDs to gene names

All target gene names are represented by ENSEMBL gene IDs here.

In [11]:
dyo_targs = pd.Series(list(set(list(itt.chain(*red_df_dyo.targets.str.split(',').values)))))

In [12]:
print("Percentage of genes in GENCODE mapping: {:.3f} ({:,} / {:,})".format(dyo_targs.isin(gencode_gene_table.gene_id).sum()/dyo_targs.shape[0], 
                                                                    dyo_targs.isin(gencode_gene_table.gene_id).sum(),dyo_targs.shape[0]))

Percentage of genes in GENCODE mapping: 0.987 (18,092 / 18,339)


In [14]:
# Export genes that are not found.
dyo_not_found = dyo_targs.loc[~dyo_targs.isin(gencode_gene_table.gene_id)]
with open("/import/kg_dyows05/moyon/Thesis/projects/merge_enhancers/generated_files/PEGASUS_missing_gene_ids.tsv",'w') as f:
    for g in dyo_not_found.unique():
        f.write(g)
        f.write('\n')

In [26]:
# Perform manually the mapping

# And load the results
with open("/import/kg_dyows05/moyon/Thesis/projects/merge_enhancers/generated_files/20181216_ensemblGrch37_ID_converted_PEGASUS.tsv",'r') as f:
    all_results = []
    new_gene = []
    for l in f:
        fields = l.strip('\n').split(', ')
        if fields[0] == '': continue
        if fields[0] == 'Old stable ID':
            if len(new_gene)>0:
                df_gene = pd.DataFrame(new_gene[1:],columns=new_gene[0])
                all_results.append(df_gene)
                new_gene = [fields]
            else:
                new_gene.append(fields)
        else:
            new_gene.append(fields)
            
    df_gene = pd.DataFrame(new_gene[1:],columns=new_gene[0])
    all_results.append(df_gene)
    
dyo_not_found_biomart_table = pd.concat([table.sort_values(by=['Release','Mapping score'],
                                                           ascending=[False,False]
                                                          ).assign(request=request_g)
                                         for request_g, table in zip(dyo_not_found.values, all_results)])

dyo_not_found_biomart_table.Release = dyo_not_found_biomart_table.Release.astype(float)

In [27]:
# Now for each requested gene, look at the best hit in the 76 version. If "retired", print it,
# otherwise store the mapping.

dyo_missing_map = {}
dyo_n_retired = 0

for missing_g_dyo, res_df in dyo_not_found_biomart_table.groupby('request'):
    #print(res_df.loc[res_df.Release=='76',:,])
    best_match = res_df.iloc[0,:]['New stable ID']
    latest_release = res_df.iloc[0,:]['Release']
    if best_match =='<retired>':
        dyo_n_retired+=1
        print("{} was retired in latest Release ({})".format(missing_g_dyo, int(latest_release)))
        
    else:
        dyo_missing_map[missing_g_dyo] = best_match.split('.')[0]
        
new_g_in_gencode = 0

new_dyo_missing_map = {}
for missing_g, new_g in dyo_missing_map.items():
    if new_g in gencode_gene_table.gene_id.values:
        new_g_in_gencode+=1
        new_dyo_missing_map[missing_g] = new_g
        
print(("Out of {} missing genes, {} were retired ; {} were re-assigned a new id ; of which {} are found in GENCODE (and thus can be kept)"
       ).format(len(dyo_not_found),dyo_n_retired, len(dyo_missing_map), new_g_in_gencode))

ENSG00000005955 was retired in latest Release (76)
ENSG00000006074 was retired in latest Release (76)
ENSG00000006075 was retired in latest Release (76)
ENSG00000006114 was retired in latest Release (76)
ENSG00000017373 was retired in latest Release (76)
ENSG00000017621 was retired in latest Release (76)
ENSG00000056661 was retired in latest Release (76)
ENSG00000068793 was retired in latest Release (76)
ENSG00000069712 was retired in latest Release (92)
ENSG00000073009 was retired in latest Release (76)
ENSG00000077809 was retired in latest Release (76)
ENSG00000083842 was retired in latest Release (76)
ENSG00000087916 was retired in latest Release (76)
ENSG00000102069 was retired in latest Release (76)
ENSG00000102080 was retired in latest Release (76)
ENSG00000108264 was retired in latest Release (76)
ENSG00000108270 was retired in latest Release (76)
ENSG00000108272 was retired in latest Release (76)
ENSG00000108278 was retired in latest Release (76)
ENSG00000108292 was retired in 

In [104]:
%%time
# Now apply this dictionary
new_targs_dyo = red_df_dyo.targets.replace(np.nan,'').apply(lambda v: ';'.join([new_dyo_missing_map.get(g,'') if g in dyo_not_found.values else g for g in v.split(',')]).strip(' ; '))
red_df_dyo['targets'] = new_targs_dyo

CPU times: user 2min 15s, sys: 864 ms, total: 2min 16s
Wall time: 2min 16s


And finally remove any region not associated to a target gene.

In [105]:
sum(red_df_dyo.targets=='')

0

In [106]:
red_df_dyo = red_df_dyo.loc[red_df_dyo.targets!='',:]

And now transform to gene names.

In [107]:
red_df_dyo['targets_names'] = red_df_dyo.targets.apply(lambda v: ';'.join(tuple(set([MAP_TO_NAMEs[g] for g in v.split(';') if g!='']))).strip(' ; '))

In [108]:
red_df_dyo.head()

,chrom,start,end,name,score,targets,targets_names
0,chr1,25639,25727,PEGASUS_chr1_1,0.01,ENSG00000187608;ENSG00000188976;ENSG0000026817...,SAMD11;HES4;ISG15;NOC2L;KLHL17;PLEKHN1;AL645608.1
1,chr1,46734,46765,PEGASUS_chr1_6,0.43,ENSG00000235249;ENSG00000186092,OR4F29;OR4F5
2,chr1,46774,46841,PEGASUS_chr1_7,0.48,ENSG00000186092,OR4F5
3,chr1,46842,46888,PEGASUS_chr1_8,0.48,ENSG00000186092,OR4F5
4,chr1,47234,47255,PEGASUS_chr1_9,0.31,ENSG00000235249,OR4F29


##### Unique non-overlapping regions

In [109]:
%%time
# Do not overlap book-ended regions
merged_df_dyo = pbt.BedTool.from_dataframe(red_df_dyo.iloc[:,[0,1,2]]).merge(d=-1).to_dataframe()
print("Shape before merge: {:,} ; shape after merge: {:,}\n\n".format(red_df_dyo.shape[0], merged_df_dyo.shape[0]))

Shape before merge: 1,308,095 ; shape after merge: 1,308,095


CPU times: user 7.52 s, sys: 564 ms, total: 8.09 s
Wall time: 10.8 s


If the option d is kept to 0, regions may be merged. But this should not be taken into account, as such situation might arise from the fact that two regions were identified as conserved, but they don't have the same depth of conservation / alignment.

##### Finale formatting

In [110]:
red_df_dyo.head()

,chrom,start,end,name,score,targets,targets_names
0,chr1,25639,25727,PEGASUS_chr1_1,0.01,ENSG00000187608;ENSG00000188976;ENSG0000026817...,SAMD11;HES4;ISG15;NOC2L;KLHL17;PLEKHN1;AL645608.1
1,chr1,46734,46765,PEGASUS_chr1_6,0.43,ENSG00000235249;ENSG00000186092,OR4F29;OR4F5
2,chr1,46774,46841,PEGASUS_chr1_7,0.48,ENSG00000186092,OR4F5
3,chr1,46842,46888,PEGASUS_chr1_8,0.48,ENSG00000186092,OR4F5
4,chr1,47234,47255,PEGASUS_chr1_9,0.31,ENSG00000235249,OR4F29


In [111]:
red_df_dyo.loc[red_df_dyo.targets_names.replace(np.nan,'')=='',:].shape

(0, 7)

In [118]:
output_df_dyo = red_df_dyo.assign(name = 'src:pegasus,'+'score:'+red_df_dyo.score.astype(str)+',targets:'+red_df_dyo.targets_names
                                 ).loc[:,['chrom','start','end','name']]

display(output_df_dyo.head(3))

pbt.BedTool.from_dataframe(output_df_dyo
                           ).to_dataframe().to_csv(
    "/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/DYOGEN/pegasus_enh.bed",
                     header=False,index=False,sep="\t")

# Don't forget to run bgzip and tabix -0 -p bed
if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/DYOGEN/pegasus_enh.bed.gz"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/DYOGEN/pegasus_enh.bed.gz")
    
if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/DYOGEN/pegasus_enh.bed.gz.tbi"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/DYOGEN/pegasus_enh.bed.gz.tbi")
    
res_call = subprocess.call('bgzip {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/DYOGEN/pegasus_enh.bed"), shell=True)
print(res_call)
# Now call tabix
res_call = subprocess.call('tabix -p bed -0 {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/DYOGEN/pegasus_enh.bed"+'.gz'), shell=True)
print(res_call)

,chrom,start,end,name
0,chr1,25639,25727,"src:pegasus,score:0.01,targets:SAMD11;HES4;ISG..."
1,chr1,46734,46765,"src:pegasus,score:0.43,targets:OR4F29;OR4F5"
2,chr1,46774,46841,"src:pegasus,score:0.48,targets:OR4F5"


0
0


### FANTOM5

In [374]:
path_fan = "/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FANTOM5//raw/enhancer_tss_associations.bed.gz"
df_fan = pd.read_table(path_fan)
display(df_fan.head())

,#chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,chromStarts
0,chr1,858252,861621,chr1:858256-858648;NM_152486;SAMD11;R:0.404;FDR:0,404,.,858452,858453,"0,0,0",2,"401,1001","0,2368"
1,chr1,894178,956888,chr1:956563-956812;NM_015658;NOC2L;R:0.202;FDR...,202,.,956687,956688,"0,0,0",2,"1001,401","0,62309"
2,chr1,901376,956888,"chr1:956563-956812;NM_001160184,NM_032129;PLEK...",422,.,956687,956688,"0,0,0",2,"1001,401","0,55111"
3,chr1,901376,1173762,"chr1:1173386-1173736;NM_001160184,NM_032129;PL...",311,.,1173561,1173562,"0,0,0",2,"1001,401","0,271985"
4,chr1,935051,942164,"chr1:941791-942135;NM_001142467,NM_021170;HES4...",187,.,941963,941964,"0,0,0",2,"1001,401","0,6712"


Here we need to extract the fields in 'name', as they represent the enhancer coordinates, targets, and scores of association.

In [375]:
tmp = df_fan.name.str.split(';',expand=True)
tmp.columns = ['reg','targets_redseqids','targets_names','R','FDR']

In [376]:
tmp.head()

,reg,targets_redseqids,targets_names,R,FDR
0,chr1:858256-858648,NM_152486,SAMD11,R:0.404,FDR:0
1,chr1:956563-956812,NM_015658,NOC2L,R:0.202,FDR:8.01154668254404e-08
2,chr1:956563-956812,"NM_001160184,NM_032129",PLEKHN1,R:0.422,FDR:0
3,chr1:1173386-1173736,"NM_001160184,NM_032129",PLEKHN1,R:0.311,FDR:0
4,chr1:941791-942135,"NM_001142467,NM_021170",HES4,R:0.187,FDR:6.32949888009368e-07


In [406]:
red_df_fan = pd.concat([tmp.reg.str.split(':|-',expand=True).rename(columns={0:'chrom',1:'start',2:'end'}),
                        tmp.targets_names,
                        tmp.R.str.split(':',expand=True).iloc[:,1].astype(float).rename('R'),
                        tmp.FDR.str.split(':',expand=True).iloc[:,1].astype(float).rename('FDR'),
                       ],axis=1)
red_df_fan['start'] = red_df_fan['start'].astype(int)
red_df_fan['end'] = red_df_fan['end'].astype(int)
red_df_fan = red_df_fan.sort_values(by=['chrom','start']).reset_index(drop=True)

display(red_df_fan.head())

,chrom,start,end,targets_names,R,FDR
0,chr1,858256,858648,SAMD11,0.404,0.000000e+00
1,chr1,941791,942135,HES4,0.187,6.329499e-07
2,chr1,945769,946034,CCNL2,0.176,3.217870e-06
3,chr1,945769,946034,ATAD3B,0.178,2.396133e-06
4,chr1,956563,956812,NOC2L,0.202,8.011547e-08


##### Genes

In [407]:
fan_targs = pd.Series(list(set(list(itt.chain(*red_df_fan.targets_names.str.split(',').values)))))
print("Percentage of genes in GENCODE mapping: {:.3f} ({:,} / {:,})".format(fan_targs.isin(gencode_gene_table.gene_name).sum()/fan_targs.shape[0], 
                                                                    fan_targs.isin(gencode_gene_table.gene_name).sum(),fan_targs.shape[0]))

Percentage of genes in GENCODE mapping: 0.920 (10,675 / 11,607)


In [408]:
fan_not_found = fan_targs.loc[~fan_targs.isin(gencode_gene_table.gene_name)]

In [409]:
%%time
# The function below looks for each gene name in the HGNC table, in synonyms or obsolete names.
# If found: associate the requested name to the updated one.

(update_gene,
 ignored_genes,
 false_positives_prev,
 false_positives_syn,
 genes_not_found) = get_updated_symbols(fan_not_found, hgnc_gene_table)

CPU times: user 34.9 s, sys: 132 ms, total: 35.1 s
Wall time: 35 s


In [410]:
print("Number of gene names that can be updated: {:,}".format(len(update_gene)))
print("Number of gene names that cannot be updated: {:,}".format(len(genes_not_found)))

Number of gene names that can be updated: 861
Number of gene names that cannot be updated: 70


In [411]:
# We still need to check that the updated version of gene names is in the GENCODE file.
finale_update_fan_g = {}
for k, v in update_gene.items():
    if v not in gencode_gene_table.gene_name.values:
        print("Updated name for {} is {}, but not found in GENCODE".format(k,v))
    else:
        finale_update_fan_g[k] = v
        
print("Finale number of genes to update: {}".format(len(finale_update_fan_g)))

Updated name for C14orf183 is LINC01599, but not found in GENCODE
Finale number of genes to update: 860


A vast majority can be updated ; all genes without updated symbol / with updated symbol not found are removed (along with the enhancer).

In [412]:
red_df_fan.head()

,chrom,start,end,targets_names,R,FDR
0,chr1,858256,858648,SAMD11,0.404,0.000000e+00
1,chr1,941791,942135,HES4,0.187,6.329499e-07
2,chr1,945769,946034,CCNL2,0.176,3.217870e-06
3,chr1,945769,946034,ATAD3B,0.178,2.396133e-06
4,chr1,956563,956812,NOC2L,0.202,8.011547e-08


In [413]:
%%time
# Now apply this dictionarry
new_targs_fan = red_df_fan.targets_names.replace(np.nan,'').apply(
                    lambda v: ';'.join([finale_update_fan_g.get(g,'') if g in fan_not_found.values else g
                                        for g in v.split(',')]).strip(' ; '))
red_df_fan['new_targets'] = new_targs_fan

CPU times: user 3.44 s, sys: 48 ms, total: 3.49 s
Wall time: 3.48 s


##### Overlapping regions

In [414]:
red_df_fan = red_df_fan.loc[~(red_df_fan['new_targets'].replace(np.nan,'')==''),:]

In [415]:
%%time
# Do not overlap book-ended regions
merged_df_fan = pbt.BedTool.from_dataframe(red_df_fan.iloc[:,[0,1,2]]).sort().merge(d=-1).to_dataframe()
print("Shape before merge: {:,} ; shape after merge: {:,}\n\n".format(red_df_fan.shape[0], merged_df_fan.shape[0]))

Shape before merge: 66,639 ; shape after merge: 27,400


CPU times: user 284 ms, sys: 1.12 s, total: 1.4 s
Wall time: 1.97 s


In [416]:
fan_new_names = red_df_fan.apply(lambda row: 'score:'+str(row['R'])+',targets:'+row['new_targets'],axis=1)
print(fan_new_names[:5])

0    score:0.404,targets:SAMD11
1      score:0.187,targets:HES4
2     score:0.176,targets:CCNL2
3    score:0.178,targets:ATAD3B
4     score:0.202,targets:NOC2L
dtype: object


In [417]:
red_df_fan.head()

,chrom,start,end,targets_names,R,FDR,new_targets
0,chr1,858256,858648,SAMD11,0.404,0.000000e+00,SAMD11
1,chr1,941791,942135,HES4,0.187,6.329499e-07,HES4
2,chr1,945769,946034,CCNL2,0.176,3.217870e-06,CCNL2
3,chr1,945769,946034,ATAD3B,0.178,2.396133e-06,ATAD3B
4,chr1,956563,956812,NOC2L,0.202,8.011547e-08,NOC2L


Below: merge the regions, and reannotate them from the original names, displayed above.

In [420]:
output_df_fan_tmp = red_df_fan.iloc[:,[0,1,2]].assign(name = fan_new_names.values
                                 ).loc[:,['chrom','start','end','name']]

output_df_fan_tmp.to_csv("/localtmp/moyon/tmp_fan.tsv",header=False,index=False,sep="\t")

res_call = subprocess.call((command_line_merge
                           ).format(inputfile="/localtmp/moyon/tmp_fan.tsv",
                                    outputfile="/localtmp/moyon/tmp_fan_MERGED.tsv",
                                    delim="\|"
                                   ), shell=True)
print(res_call)

assert res_call==0, "Error while generating the merged version of the file."

finale_merged_df_fan = pd.read_table("/localtmp/moyon/tmp_fan_MERGED.tsv",header=None,names=['chrom','start','end','name'])

display(finale_merged_df_fan.head())

0


,chrom,start,end,name
0,chr1,858256,858648,"score:0.404,targets:SAMD11"
1,chr1,941791,942135,"score:0.187,targets:HES4"
2,chr1,945769,946034,"score:0.178,targets:ATAD3B|score:0.176,targets..."
3,chr1,956563,956812,"score:0.422,targets:PLEKHN1|score:0.202,target..."
4,chr1,1005293,1005547,"score:0.236,targets:HES4"


##### Finale formatting

In [421]:
output_df_fan = finale_merged_df_fan

display(output_df_fan.head(3))

pbt.BedTool.from_dataframe(output_df_fan
                           ).sort().to_dataframe().to_csv(
    "/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FANTOM5/fantom5_enh.bed",
                     header=False,index=False,sep="\t")


if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FANTOM5/fantom5_enh.bed.gz"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FANTOM5/fantom5_enh.bed.gz")
    
if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FANTOM5/fantom5_enh.bed.gz.tbi"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FANTOM5/fantom5_enh.bed.gz.tbi")
    

res_call = subprocess.call('bgzip {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FANTOM5/fantom5_enh.bed"), shell=True)
print(res_call)
# Now call tabix
res_call = subprocess.call('tabix -p bed -0 {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FANTOM5/fantom5_enh.bed.gz"), shell=True)
print(res_call)
# Don't forget to run bgzip and tabix -0 -p bed

,chrom,start,end,name
0,chr1,858256,858648,"score:0.404,targets:SAMD11"
1,chr1,941791,942135,"score:0.187,targets:HES4"
2,chr1,945769,946034,"score:0.178,targets:ATAD3B|score:0.176,targets..."


0
0


### GENEHANCER

In [50]:
path_genehancer = "/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/GeneHancer/raw/hg19.GeneHancer_version_4_4.bed"
df_genh = pd.read_table(path_genehancer, header=None, names=['chrom','start','end','source','score','strand','annot'])
display(df_genh.head())

,chrom,start,end,source,score,strand,annot
0,chr2,70244933,70245132,GeneHancer,0.52,.,genehancer_id=GH02F070017;connected_gene=PCBP1...
1,chr18,46119061,46119210,GeneHancer,0.96,.,genehancer_id=GH18F048592;connected_gene=ENSG0...
2,chr5,167739362,167746222,GeneHancer,1.77,.,genehancer_id=GH05F168312;connected_gene=WWC1;...
3,chr12,47598361,47613394,GeneHancer,1.28,.,genehancer_id=GH12F047204;connected_gene=RPAP3...
4,chr21,41001999,41006993,GeneHancer,1.29,.,genehancer_id=GH21F039630;connected_gene=B3GAL...


In [51]:
df_genh.annot[0].split(';')[1:]

['connected_gene=PCBP1-AS1',
 'score=12.90',
 'connected_gene=ASPRV1',
 'score=11.69',
 'connected_gene=PIR47967',
 'score=0.29',
 'connected_gene=RN7SL470P',
 'score=0.20']

We need to reformat the annotations.

In [52]:
def reformat_genhancer(annot):
    new_cols = ('src','targets','score')
    sp = annot.split(';')
    src = sp[0].split('=')[1]
    sp = [v.split('=')[1] for v in sp[1:]]
    
    all_groups = []
    for i in range(0,len(sp),2):
        group = ';'.join([':'.join(v) for v in zip(new_cols,[src]+sp[i:i+2])])
        all_groups.append(group)
        
    return '|'.join(all_groups)        

In [53]:
%%time
genh_newannots = df_genh.annot.apply(reformat_genhancer)

CPU times: user 3.65 s, sys: 52 ms, total: 3.7 s
Wall time: 3.71 s


In [54]:
genh_newannots.head()

0    src:GH02F070017;targets:PCBP1-AS1;score:12.90|...
1    src:GH18F048592;targets:ENSG00000278983;score:...
2    src:GH05F168312;targets:WWC1;score:12.82|src:G...
3    src:GH12F047204;targets:RPAP3;score:17.58|src:...
4    src:GH21F039630;targets:B3GALT5-AS1;score:46.8...
Name: annot, dtype: object

In [56]:
genh_newannots[0]

'src:GH02F070017;targets:PCBP1-AS1;score:12.90|src:GH02F070017;targets:ASPRV1;score:11.69|src:GH02F070017;targets:PIR47967;score:0.29|src:GH02F070017;targets:RN7SL470P;score:0.20'

##### Gene names

Again we need to update the genes so that they match GENCODE ones.

In [57]:
%%time
genh_missing_ENS = []
genh_missing_sym = []

genh_all_genes = []

for annot in genh_newannots:
    groups = annot.split('|')
    for g in groups:
        targs = dataframes.get_field_kv_pairs_list(g,';',':','targets')
        
        
        if targs=='':
            continue
            
        for t in targs.split(','):
            genh_all_genes.append(t)
            
            if t.startswith('ENS'):
                if t not in gencode_gene_table.gene_id.values:
                    genh_missing_ENS.append(t)
                    
            else:
                if t not in gencode_gene_table.gene_name.values:
                    genh_missing_sym.append(t)

CPU times: user 16min 43s, sys: 296 ms, total: 16min 43s
Wall time: 16min 43s


In [58]:
set_genh_genes = list(set(genh_all_genes))

In [59]:
genh_missing_ENS = list(set(genh_missing_ENS))

In [60]:
genh_missing_sym = list(set(genh_missing_sym))

In [61]:
print("Total nb of Genhancers genes: {:,}\nNb missing on Ensembl Ids: {:,}\nNb missing from symbols: {:,}".format(len(set_genh_genes), len(genh_missing_ENS), len(genh_missing_sym)))

Total nb of Genhancers genes: 95,408
Nb missing on Ensembl Ids: 2,405
Nb missing from symbols: 51,644


The total number of genes is weird ... This might explain that I loose half of them.

In fact, they put some of their annotated GeneCards genes. I think I can safely ignore these genes.

In [62]:
N_gc=0
N_ot=0
for mg in genh_missing_sym:
    if mg.startswith('GC'):
        N_gc+=1
    else:
        N_ot+=1
print("Out of the {} missing genes, {} are annotated from GeneCard, and {} from other resources...".format(len(genh_missing_sym), N_gc, N_ot))

Out of the 51644 missing genes, 29737 are annotated from GeneCard, and 21907 from other resources...


After a few test : some of these in the "other resources" may be found in HGNC, but fail to be found in GENCODE (even using the ensemblID). I choose to ignore them too.

Also, in the other type of missing genes, most are miRNA, piRNA, or ncRNA. I can ignore them too.

In [63]:
%%time

(genh_updated_sym,
 genh_ignored_genes,
 genh_false_positives_prev,
 genh_false_positives_syn,
 genh_genes_not_found) = get_updated_symbols([mg for mg in genh_missing_sym if not mg.startswith('GC')], hgnc_gene_table)

CPU times: user 19min 11s, sys: 2.52 s, total: 19min 13s
Wall time: 19min 16s


In [64]:
print("Number of gene names that can be updated: {:,}".format(len(genh_updated_sym)))
print("Number of gene names that cannot be updated: {:,}".format(len(genh_genes_not_found)))

Number of gene names that can be updated: 640
Number of gene names that cannot be updated: 21,266


In [83]:
missing_genes_weird = []
other_missing_genes = []
for g in genh_genes_not_found:
    if g.startswith('PIR') or g.startswith('LOC') or g.startswith('MIR') or g.startswith('SNO'):
        missing_genes_weird.append(g)
    else:
        other_missing_genes.append(g)
        
        
print("# missing genes with name indicating an unknown locus, MIR, or PIR, or SNO: {:,}".format(len(missing_genes_weird)))
print("# missing genes with name appearing to be 'normal': {:,}".format(len(other_missing_genes)))

# missing genes with name indicating an unknown locus, MIR, or PIR, or SNO: 19,499
# missing genes with name appearing to be 'normal': 1,767


In [78]:
with open("./generated_files/genehancer_missing_IDs.tsv",'w') as f:
    for g in genh_missing_ENS:
        f.write(g)
        f.write('\n')

In [79]:
# And load the results
with open("./generated_files/20181217_ensemblGrch37_ID_converted_GENENHANCER.tsv",'r') as f:
    all_results = []
    new_gene = []
    for l in f:
        fields = l.strip('\n').split(', ')
        if fields[0] == '': continue
        if fields[0] == 'Old stable ID':
            if len(new_gene)>0:
                df_gene = pd.DataFrame(new_gene[1:],columns=new_gene[0])
                all_results.append(df_gene)
                new_gene = [fields]
            else:
                new_gene.append(fields)
        else:
            new_gene.append(fields)
            
    df_gene = pd.DataFrame(new_gene[1:],columns=new_gene[0])
    all_results.append(df_gene)
    
genh_not_found_biomart_table = pd.concat([table.sort_values(by=['Release','Mapping score'],
                                                           ascending=[False,False]
                                                          ).assign(request=request_g)
                                         for request_g, table in zip(genh_missing_ENS, all_results)])

genh_not_found_biomart_table.Release = genh_not_found_biomart_table.Release.astype(float)

In [87]:
genh_not_found_biomart_table['Old stable ID'].unique().shape

(960,)

In [92]:
# Those are the genes that are in GENENHANCER but not found in GENCODE, but not even remapped to an old Ensembl ID by BioMart...
for g in genh_missing_ENS:
    if g not in genh_not_found_biomart_table['Old stable ID'].unique():
        print(g)

ENSG00000228404
ENSG00000258297
ENSG00000253065
ENSG00000253047
ENSG00000280389
ENSG00000278482
ENSG00000279131
ENSG00000273750
ENSG00000258941
ENSG00000200888
ENSG00000201279
ENSG00000248317
ENSG00000200742
ENSG00000207024
ENSG00000278413
ENSG00000206976
ENSG00000223080
ENSG00000206679
ENSG00000263823
ENSG00000202035
ENSG00000231013
ENSG00000281527
ENSG00000275908
ENSG00000227824
ENSG00000206741
ENSG00000200492
ENSG00000223271
ENSG00000283432
ENSG00000254662
ENSG00000260651
ENSG00000199490
ENSG00000206840
ENSG00000254281
ENSG00000200855
ENSG00000212532
ENSG00000201741
ENSG00000260521
ENSG00000207282
ENSG00000275287
ENSG00000206995
ENSG00000279534
ENSG00000252894
ENSG00000274848
ENSG00000271762
ENSG00000199832
ENSG00000254757
ENSG00000251099
ENSG00000281042
ENSG00000277677
ENSG00000257286
ENSG00000241539
ENSG00000281331
ENSG00000264562
ENSG00000261496
ENSG00000276494
ENSG00000266973
ENSG00000274197
ENSG00000269921
ENSG00000200506
ENSG00000207101
ENSG00000273701
ENSG00000202014
ENSG0000

limit_output extension: Maximum message size of 10000 exceeded with 10382 characters

In [80]:
# Now for each requested gene, look at the best hit in the 76 version. If "retired", print it,
# otherwise store the mapping.

genh_ENS_missing_map = {}
genh_ENS_n_retired = 0

for missing_g, res_df in genh_not_found_biomart_table.groupby('request'):
    #print(res_df.loc[res_df.Release=='76',:,])
    best_match = res_df.iloc[0,:]['New stable ID']
    latest_release = res_df.iloc[0,:]['Release']
    if best_match =='<retired>':
        genh_ENS_n_retired+=1
        print("{} was retired in latest Release ({})".format(missing_g, int(latest_release)))
        
    else:
        genh_ENS_missing_map[missing_g] = best_match.split('.')[0]
        
genh_new_g_in_gencode = 0

new_genh_ENS_missing_map = {}
for missing_g, new_g in genh_ENS_missing_map.items():
    if new_g in gencode_gene_table.gene_id.values:
        genh_new_g_in_gencode+=1
        new_genh_ENS_missing_map[missing_g] = new_g
     

ENSG00000150076 was retired in latest Release (88)
ENSG00000182109 was retired in latest Release (92)
ENSG00000187695 was retired in latest Release (92)
ENSG00000188477 was retired in latest Release (88)
ENSG00000205662 was retired in latest Release (94)
ENSG00000205663 was retired in latest Release (94)
ENSG00000215447 was retired in latest Release (92)
ENSG00000223401 was retired in latest Release (92)
ENSG00000223473 was retired in latest Release (88)
ENSG00000223562 was retired in latest Release (88)
ENSG00000224235 was retired in latest Release (92)
ENSG00000224441 was retired in latest Release (92)
ENSG00000224460 was retired in latest Release (88)
ENSG00000224850 was retired in latest Release (88)
ENSG00000224911 was retired in latest Release (88)
ENSG00000225302 was retired in latest Release (92)
ENSG00000225541 was retired in latest Release (94)
ENSG00000225704 was retired in latest Release (94)
ENSG00000225745 was retired in latest Release (92)
ENSG00000226434 was retired in 

limit_output extension: Maximum message size of 10000 exceeded with 13463 characters

In [81]:
print(("Out of {} missing genes, {} were retired ; {} were re-assigned a new id ; of which {} are found in GENCODE (and thus can be kept)"
       ).format(len(genh_missing_ENS),genh_ENS_n_retired, len(genh_ENS_missing_map), genh_new_g_in_gencode))

Out of 2405 missing genes, 313 were retired ; 373 were re-assigned a new id ; of which 0 are found in GENCODE (and thus can be kept)


So we only update the genes from the genh_updated_sym table. All other are replaced by empty strings, and the associated enhancers will be removed.

In [96]:
def genhancer_update_annots(annot):
    N_removed_interactions = 0
    N_total_interactions =0

    newgroups = []
    groups = annot.split('|')

    for g in groups:
        N_total_interactions +=1

        fields = dataframes.transform_to_dict(g,';',':')

        targs = fields['targets']     
        newtargs = []
        if targs!='':
            for t in targs.split(','):
                if t.startswith('ENS'):
                    if t in genh_missing_ENS:
                        newt = ''
                    else:
                        newt= MAP_TO_NAMEs.get(t,'')
                        
                else:
                    newt = genh_updated_sym.get(t, '') if t in genh_missing_sym else t
                    
                if newt!='':
                    newtargs.append(newt)

        else:
            N_removed_interactions+=1
            continue

        # Now update the group.
        if len(newtargs)==0:
            N_removed_interactions+=1
            continue
        else:
            newtargs = ';'.join(newtargs)
            fields['targets'] = newtargs
            newgroups.append(dataframes.convert_keyvalue_pairs_to_str(fields.items(),',',':'))

    # And now rejoin the newgroups.
    if len(newgroups)==0:
        # This enhancer has no interactions anymore and will be removed.
        newgroups = ''

    else:
        newgroups = '|'.join(newgroups)
        
    return (newgroups, N_removed_interactions, N_total_interactions)

In [97]:
%%time
N_removed_interactions = 0
N_total_interactions =0

pool = multiprocessing.Pool(6)
updated_genh_newannots_wcounts = pool.map(genhancer_update_annots, genh_newannots)
pool.close()
pool.join()

N_removed_interactions = np.array([v[1] for v in updated_genh_newannots_wcounts]).sum()
N_total_interactions = np.array([v[2] for v in updated_genh_newannots_wcounts]).sum()

updated_genh_newannots = np.array([v[0] for v in updated_genh_newannots_wcounts])

CPU times: user 2.28 s, sys: 6.22 s, total: 8.5 s
Wall time: 7min 54s


In [98]:
print("Out of {:,} interactions, {:,} were removed because of lack of target gene".format(N_total_interactions,N_removed_interactions))

Out of 817,390 interactions, 289,570 were removed because of lack of target gene


In [567]:
print("Out of {:,} interactions, {:,} were removed because of lack of target gene".format(N_total_interactions,N_removed_interactions))

Out of 817,390 interactions, 275,675 were removed because of lack of target gene


In [99]:
df_genh['new_annot']=updated_genh_newannots
genh_filtered = df_genh.new_annot==''
print("Number of enhancers initially: {:,}\nNumber that are removed because of no annot: {:,}".format(df_genh.shape[0], genh_filtered.sum()))

Number of enhancers initially: 217,695
Number that are removed because of no annot: 32,885


In [100]:
df_genh = df_genh.loc[~genh_filtered,:]

In [ ]:
#genomeCov_genh = pbt.BedTool.from_dataframe(df_genh.iloc[:,[0,1,2]]).genome_coverage(g='/users/ldog/moyon/Thesis/RegulationData/hg19/hg19.chrom.sizes')

##### Merging

In [101]:
%%time
# Do not overlap book-ended regions
merged_df_genh = pbt.BedTool.from_dataframe(df_genh.iloc[:,[0,1,2]]).sort().merge(d=-1).to_dataframe()
print("Shape before merge: {:,} ; shape after merge: {:,}".format(df_genh.shape[0], merged_df_genh.shape[0]))

Shape before merge: 184,810 ; shape after merge: 184,643
CPU times: user 1.18 s, sys: 268 ms, total: 1.45 s
Wall time: 4.8 s


I will keep these as is, and let a variant intersect multiple regions.

Then the best scoring enhancer is kept, with associated targets... (not ideal...)

##### Exporting

In [102]:
df_genh.head()

,chrom,start,end,source,score,strand,annot,new_annot
0,chr2,70244933,70245132,GeneHancer,0.52,.,genehancer_id=GH02F070017;connected_gene=PCBP1...,"src:GH02F070017,targets:PCBP1-AS1,score:12.90|..."
1,chr18,46119061,46119210,GeneHancer,0.96,.,genehancer_id=GH18F048592;connected_gene=ENSG0...,"src:GH18F048592,targets:AC048380.2,score:0.29|..."
2,chr5,167739362,167746222,GeneHancer,1.77,.,genehancer_id=GH05F168312;connected_gene=WWC1;...,"src:GH05F168312,targets:WWC1,score:12.82"
3,chr12,47598361,47613394,GeneHancer,1.28,.,genehancer_id=GH12F047204;connected_gene=RPAP3...,"src:GH12F047204,targets:RPAP3,score:17.58|src:..."
4,chr21,41001999,41006993,GeneHancer,1.29,.,genehancer_id=GH21F039630;connected_gene=B3GAL...,"src:GH21F039630,targets:B3GALT5-AS1,score:46.8..."


In [103]:
output_df_genh = df_genh.assign(name = df_genh.new_annot
                                 ).loc[:,['chrom','start','end','source', 'score', 'strand', 'name']]

display(output_df_genh.head(3))

pbt.BedTool.from_dataframe(output_df_genh).sort(
                           ).to_dataframe().to_csv(
    "/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/GeneHancer/genehancer.bed",
                     header=False,index=False,sep="\t")

if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/GeneHancer/genehancer.bed.gz"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/GeneHancer/genehancer.bed.gz")
    
if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/GeneHancer/genehancer.bed.gz.tbi"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/GeneHancer/genehancer.bed.gz.tbi")
    

res_call = subprocess.call('bgzip {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/GeneHancer/genehancer.bed"), shell=True)
print(res_call)
# Now call tabix
res_call = subprocess.call('tabix -p bed -0 {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/GeneHancer/genehancer.bed.gz"), shell=True)
print(res_call)

,chrom,start,end,source,score,strand,name
0,chr2,70244933,70245132,GeneHancer,0.52,.,"src:GH02F070017,targets:PCBP1-AS1,score:12.90|..."
1,chr18,46119061,46119210,GeneHancer,0.96,.,"src:GH18F048592,targets:AC048380.2,score:0.29|..."
2,chr5,167739362,167746222,GeneHancer,1.77,.,"src:GH05F168312,targets:WWC1,score:12.82"


0
0


### FOCS

#### FOCS FANTOM

In [348]:
focs = pd.read_table("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/raw/fantom_interactions.txt")
display(focs.columns)
print("\n")
red_df_focs = focs.loc[:,['e_chr','e_start','e_end','e_strand','enh_id','sym_id','ens_id']].sort_values(by=['e_chr','e_start']).reset_index(drop=True)
display(red_df_focs.head())

Index(['p_chr', 'p_start', 'p_end', 'p_strand', 'e_chr', 'e_start', 'e_end',
       'e_strand', 'prom_id', 'enh_id', 'ep_dist', 'sym_id', 'ent_id',
       'ens_id', 'ref_id'],
      dtype='object')

,e_chr,e_start,e_end,e_strand,enh_id,sym_id,ens_id
0,chr1,858256,858648,*,e_chr1:858257_858648,PERM1,ENSG00000187642
1,chr1,868565,868684,*,e_chr1:868566_868684,HES4,ENSG00000188290
2,chr1,918449,918555,*,e_chr1:918450_918555,PERM1,ENSG00000187642
3,chr1,936652,937138,*,e_chr1:936653_937138,HES4,ENSG00000188290
4,chr1,936652,937138,*,e_chr1:936653_937138,HES4,ENSG00000188290


In [247]:
%%time

total_genes_focs = pd.Series(list(itt.chain(*focs.sym_id.replace(np.nan,'').str.split(';').values))).unique()

focs_missing = []

for g in total_genes_focs:
    if g not in gencode_gene_table.gene_name.values:
        focs_missing.append(g)
        
focs_missing = pd.Series(focs_missing)
        
print("Percentage of genes in GENCODE mapping: {:.3f} ({:,} / {:,})".format((len(total_genes_focs)-len(focs_missing))/len(total_genes_focs), 
                                                                    (len(total_genes_focs)-len(focs_missing)),len(total_genes_focs)))


(focs_update_symbols,
 focs_ignored_genes,
 focs_false_positives_prev,
 focs_false_positives_syn,
 focs_genes_not_found) = get_updated_symbols(focs_missing, hgnc_gene_table)

print("Number of gene names that can be updated: {:,}".format(len(focs_update_symbols)))
print("Number of gene names that cannot be updated: {:,}".format(len(focs_genes_not_found)))

# We still need to check that the updated version of gene names is in the GENCODE file.
finale_update_focs_g = {}
for k, v in focs_update_symbols.items():
    if v not in gencode_gene_table.gene_name.values:
        continue
    else:
        finale_update_focs_g[k] = v
        
print("Finale number of genes to update: {:,} ({:,} could have an update, but it's not found in GENCODE.)\n\n".format(len(finale_update_focs_g),(len(focs_update_symbols)-len(finale_update_focs_g))))

Percentage of genes in GENCODE mapping: 0.986 (7,120 / 7,223)
Number of gene names that can be updated: 100
Number of gene names that cannot be updated: 2
Finale number of genes to update: 100 (0 could have an update, but it's not found in GENCODE.)


CPU times: user 9.66 s, sys: 40 ms, total: 9.7 s
Wall time: 9.7 s


In [248]:
%%time
# Now apply this dictionarry
new_targs_focs = red_df_focs.sym_id.replace(np.nan,'').apply(
                    lambda v: ';'.join([finale_update_focs_g.get(g,'') if g in focs_missing.values else g
                                        for g in v.split(',')]).strip(' ; '))
red_df_focs['new_targets'] = new_targs_focs

red_df_focs = red_df_focs.loc[~(red_df_focs['new_targets'].replace(np.nan,'')==''),:]

CPU times: user 628 ms, sys: 36 ms, total: 664 ms
Wall time: 660 ms


Below is the part where I check if there's regions that overlap in the bed file. I merge them if so.

In [249]:
%%time
# Do not overlap book-ended regions
merged_df_focs = pbt.BedTool.from_dataframe(red_df_focs.iloc[:,[0,1,2]]).sort().merge(d=-1).to_dataframe()
print("Shape before merge: {:,} ; shape after merge: {:,}\n\n".format(red_df_focs.shape[0], merged_df_focs.shape[0]))

Shape before merge: 35,385 ; shape after merge: 16,477


CPU times: user 176 ms, sys: 284 ms, total: 460 ms
Wall time: 878 ms


This means that I need to perform a merging of regions...

Since there's no particular score associated to the enhancer/target predictions, I can simply assign the "new targets" names as the "name" of each regions ; then I perform the merging with bedops, and new regions will be named as the accumulation of the target names.

In [261]:
output_df_focs_tmp = red_df_focs.assign(name = red_df_focs.new_targets
                                 ).loc[:,['e_chr','e_start','e_end','name']]

output_df_focs_tmp.to_csv("/localtmp/moyon/tmp_focs_fantom.tsv",header=False,index=False,sep="\t")

res_call = subprocess.call((command_line_merge
                           ).format(inputfile="/localtmp/moyon/tmp_focs_fantom.tsv",
                                    outputfile="/localtmp/moyon/tmp_MERGED_focs_fantom.tsv",
                                    delim="\|"
                                   ), shell=True)
print(res_call)

assert res_call==0, "Error while generating the merged version of the file."

finale_merged_df_focs = pd.read_table("/localtmp/moyon/tmp_MERGED_focs_fantom.tsv",header=None,names=['chrom','start','end','name'])

display(finale_merged_df_focs.head())

0


,chrom,start,end,name
0,chr1,858256,858648,PERM1
1,chr1,868565,868684,HES4
2,chr1,918449,918555,PERM1
3,chr1,936652,937138,ISG15|HES4|HES4
4,chr1,956563,956812,PLEKHN1|AGRN|AGRN|HES4|HES4


Here we sort-of have the possibility to compute a score : the number of times a gene appears in the list corresponds to the number of overlapping enhancers that were seen as associated to that gene.

In [351]:
%%time
merged_df_focs_names = finale_merged_df_focs['name'].apply(lambda v: from_list_genes_to_kv_pairs(v))
display(print(merged_df_focs_names.iloc[:5]))
print("\n")

0                                score:1,targets:PERM1
1                                 score:1,targets:HES4
2                                score:1,targets:PERM1
3           score:1,targets:ISG15|score:2,targets:HES4
4    score:1,targets:PLEKHN1|score:2,targets:HES4;AGRN
Name: name, dtype: object


None



CPU times: user 1min 29s, sys: 376 ms, total: 1min 30s
Wall time: 1min 30s


In [347]:
output_df_focs = finale_merged_df_focs.assign(name=merged_df_focs_names.values
                                 ).loc[:,['chrom','start','end','name']]

display(output_df_focs.head(3))

pbt.BedTool.from_dataframe(output_df_focs
                           ).sort().to_dataframe().to_csv(
    "/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/fantom5_enh.bed",
                     header=False,index=False,sep="\t")


if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/fantom5_enh.bed.gz"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/fantom5_enh.bed.gz")
    
if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/fantom5_enh.bed.gz.tbi"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/fantom5_enh.bed.gz.tbi")
    

res_call = subprocess.call('bgzip {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/fantom5_enh.bed"), shell=True)
print(res_call)
# Now call tabix
res_call = subprocess.call('tabix -p bed -0 {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/fantom5_enh.bed.gz"), shell=True)
print(res_call)
# Don't forget to run bgzip and tabix -0 -p bed

,chrom,start,end,name
0,chr1,858256,858648,"score:1,targets:PERM1"
1,chr1,868565,868684,"score:1,targets:HES4"
2,chr1,918449,918555,"score:1,targets:PERM1"


0
0


#### FOCS GROSEQ

In [360]:
focs = pd.read_table("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/raw/groseq_interactions.txt")
display(focs.columns)
print("\n")
red_df_focs = focs.loc[:,['e_chr','e_start','e_end','e_strand','enh_id','sym_id','ens_id']].sort_values(by=['e_chr','e_start']).reset_index(drop=True)
red_df_focs['e_start'] = red_df_focs.e_start.astype(int)
red_df_focs['e_end'] = red_df_focs.e_end.astype(int)
display(red_df_focs.head())

Index(['p_chr', 'p_start', 'p_end', 'p_strand', 'e_chr', 'e_start', 'e_end',
       'e_strand', 'prom_id', 'enh_id', 'ep_dist', 'sym_id', 'ent_id',
       'ens_id', 'ref_id'],
      dtype='object')

,e_chr,e_start,e_end,e_strand,enh_id,sym_id,ens_id
0,chr1,836500,837502,*,e_chr1:836501_837502,LOC100130417,ENSG00000223764
1,chr1,872650,873252,*,e_chr1:872651_873252,SAMD11,ENSG00000187634
2,chr1,873350,873602,*,e_chr1:873351_873602,PLEKHN1,NaN
3,chr1,873350,873602,*,e_chr1:873351_873602,SAMD11,ENSG00000187634
4,chr1,875050,875252,*,e_chr1:875051_875252,SAMD11,ENSG00000187634


In [361]:
%%time

total_genes_focs = pd.Series(list(itt.chain(*focs.sym_id.replace(np.nan,'').str.split(';').values))).unique()

focs_missing = []

for g in total_genes_focs:
    if g not in gencode_gene_table.gene_name.values:
        focs_missing.append(g)
        
focs_missing = pd.Series(focs_missing)
        
print("Percentage of genes in GENCODE mapping: {:.3f} ({:,} / {:,})".format((len(total_genes_focs)-len(focs_missing))/len(total_genes_focs), 
                                                                    (len(total_genes_focs)-len(focs_missing)),len(total_genes_focs)))


(focs_update_symbols,
 focs_ignored_genes,
 focs_false_positives_prev,
 focs_false_positives_syn,
 focs_genes_not_found) = get_updated_symbols(focs_missing, hgnc_gene_table)

print("Number of gene names that can be updated: {:,}".format(len(focs_update_symbols)))
print("Number of gene names that cannot be updated: {:,}".format(len(focs_genes_not_found)))

# We still need to check that the updated version of gene names is in the GENCODE file.
finale_update_focs_g = {}
for k, v in focs_update_symbols.items():
    if v not in gencode_gene_table.gene_name.values:
        continue
    else:
        finale_update_focs_g[k] = v
        
print("Finale number of genes to update: {:,} ({:,} could have an update, but it's not found in GENCODE.)\n\n".format(len(finale_update_focs_g),(len(focs_update_symbols)-len(finale_update_focs_g))))

Percentage of genes in GENCODE mapping: 0.915 (5,745 / 6,276)
Number of gene names that can be updated: 245
Number of gene names that cannot be updated: 285
Finale number of genes to update: 244 (1 could have an update, but it's not found in GENCODE.)


CPU times: user 25.3 s, sys: 72 ms, total: 25.3 s
Wall time: 25.4 s


In [362]:
%%time
# Now apply this dictionarry
new_targs_focs = red_df_focs.sym_id.replace(np.nan,'').apply(
                    lambda v: ';'.join([finale_update_focs_g.get(g,'') if g in focs_missing.values else g
                                        for g in v.split(',')]).strip(' ; '))
red_df_focs['new_targets'] = new_targs_focs

red_df_focs = red_df_focs.loc[~(red_df_focs['new_targets'].replace(np.nan,'')==''),:]

CPU times: user 1.16 s, sys: 136 ms, total: 1.29 s
Wall time: 1.19 s


Below is the part where I check if there's regions that overlap in the bed file. I merge them if so.

In [363]:
%%time
# Do not overlap book-ended regions
merged_df_focs = pbt.BedTool.from_dataframe(red_df_focs.iloc[:,[0,1,2]]).sort().merge(d=-1).to_dataframe()
print("Shape before merge: {:,} ; shape after merge: {:,}\n\n".format(red_df_focs.shape[0], merged_df_focs.shape[0]))

Shape before merge: 21,430 ; shape after merge: 19,771


CPU times: user 120 ms, sys: 316 ms, total: 436 ms
Wall time: 876 ms


This means that I need to perform a merging of regions...

Since there's no particular score associated to the enhancer/target predictions, I can simply assign the "new targets" names as the "name" of each regions ; then I perform the merging with bedops, and new regions will be named as the accumulation of the target names.

In [364]:
output_df_focs_tmp = red_df_focs.assign(name = red_df_focs.new_targets
                                 ).loc[:,['e_chr','e_start','e_end','name']]

output_df_focs_tmp.to_csv("/localtmp/moyon/tmp_focs_groseq.tsv",header=False,index=False,sep="\t")

res_call = subprocess.call((command_line_merge
                           ).format(inputfile="/localtmp/moyon/tmp_focs_groseq.tsv",
                                    outputfile="/localtmp/moyon/tmp_MERGED_focs_groseq.tsv",
                                    delim="\|"
                                   ), shell=True)
print(res_call)

assert res_call==0, "Error while generating the merged version of the file."

finale_merged_df_focs = pd.read_table("/localtmp/moyon/tmp_MERGED_focs_groseq.tsv",header=None,names=['chrom','start','end','name'])

display(finale_merged_df_focs.head())

0


,chrom,start,end,name
0,chr1,872650,873252,SAMD11
1,chr1,873350,873602,PLEKHN1|SAMD11
2,chr1,875050,875252,SAMD11
3,chr1,881400,881502,PLEKHN1
4,chr1,891050,891252,PLEKHN1


Here we sort-of have the possibility to compute a score : the number of times a gene appears in the list corresponds to the number of overlapping enhancers that were seen as associated to that gene.

In [365]:
%%time
merged_df_focs_names = finale_merged_df_focs['name'].apply(lambda v: from_list_genes_to_kv_pairs(v))
display(merged_df_focs_names.iloc[:5])
print("\n")

0            score:1,targets:SAMD11
1    score:1,targets:SAMD11;PLEKHN1
2            score:1,targets:SAMD11
3           score:1,targets:PLEKHN1
4           score:1,targets:PLEKHN1
Name: name, dtype: object


None



CPU times: user 1min 47s, sys: 396 ms, total: 1min 48s
Wall time: 1min 48s


In [366]:
output_df_focs = finale_merged_df_focs.assign(name=merged_df_focs_names.values
                                 ).loc[:,['chrom','start','end','name']]

display(output_df_focs.head(3))

pbt.BedTool.from_dataframe(output_df_focs
                           ).sort().to_dataframe().to_csv(
    "/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/groseq_enh.bed",
                     header=False,index=False,sep="\t")


if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/groseq_enh.bed.gz"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/groseq_enh.bed.gz")
    
if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/groseq_enh.bed.gz.tbi"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/groseq_enh.bed.gz.tbi")
    

res_call = subprocess.call('bgzip {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/groseq_enh.bed"), shell=True)
print(res_call)
# Now call tabix
res_call = subprocess.call('tabix -p bed -0 {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/groseq_enh.bed.gz"), shell=True)
print(res_call)
# Don't forget to run bgzip and tabix -0 -p bed

,chrom,start,end,name
0,chr1,872650,873252,"score:1,targets:SAMD11"
1,chr1,873350,873602,"score:1,targets:SAMD11;PLEKHN1"
2,chr1,875050,875252,"score:1,targets:SAMD11"


0
0


#### FOCS ROADMAP

In [367]:
focs = pd.read_table("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/raw/roadmap_interactions.txt")
display(focs.columns)
print("\n")
red_df_focs = focs.loc[:,['e_chr','e_start','e_end','e_strand','enh_id','sym_id','ens_id']].sort_values(by=['e_chr','e_start']).reset_index(drop=True)
red_df_focs['e_start'] = red_df_focs.e_start.astype(int)
red_df_focs['e_end'] = red_df_focs.e_end.astype(int)
display(red_df_focs.head())

Index(['p_chr', 'p_start', 'p_end', 'p_strand', 'e_chr', 'e_start', 'e_end',
       'e_strand', 'prom_id', 'enh_id', 'ep_dist', 'sym_id', 'ent_id',
       'ens_id', 'ref_id'],
      dtype='object')

,e_chr,e_start,e_end,e_strand,enh_id,sym_id,ens_id
0,chr1,752447,752795,*,e_chr1:752448_752795,AL669831.1;AL669831.5,ENSG00000228327;ENSG00000237491
1,chr1,753344,753691,*,e_chr1:753345_753691,AL669831.1;AL669831.5,ENSG00000228327;ENSG00000237491
2,chr1,773729,773958,*,e_chr1:773730_773958,AL669831.1;AL669831.5,ENSG00000228327;ENSG00000237491
3,chr1,778091,778515,*,e_chr1:778092_778515,LINC00115;LINC01128,ENSG00000225880;ENSG00000228794
4,chr1,779960,780218,*,e_chr1:779961_780218,AL669831.1;AL669831.5,ENSG00000228327;ENSG00000237491


In [368]:
%%time

total_genes_focs = pd.Series(list(itt.chain(*focs.sym_id.replace(np.nan,'').str.split(';').values))).unique()

focs_missing = []

for g in total_genes_focs:
    if g not in gencode_gene_table.gene_name.values:
        focs_missing.append(g)
        
focs_missing = pd.Series(focs_missing)
        
print("Percentage of genes in GENCODE mapping: {:.3f} ({:,} / {:,})".format((len(total_genes_focs)-len(focs_missing))/len(total_genes_focs), 
                                                                    (len(total_genes_focs)-len(focs_missing)),len(total_genes_focs)))


(focs_update_symbols,
 focs_ignored_genes,
 focs_false_positives_prev,
 focs_false_positives_syn,
 focs_genes_not_found) = get_updated_symbols(focs_missing, hgnc_gene_table)

print("Number of gene names that can be updated: {:,}".format(len(focs_update_symbols)))
print("Number of gene names that cannot be updated: {:,}".format(len(focs_genes_not_found)))

# We still need to check that the updated version of gene names is in the GENCODE file.
finale_update_focs_g = {}
for k, v in focs_update_symbols.items():
    if v not in gencode_gene_table.gene_name.values:
        continue
    else:
        finale_update_focs_g[k] = v
        
print("Finale number of genes to update: {:,} ({:,} could have an update, but it's not found in GENCODE.)\n\n".format(len(finale_update_focs_g),(len(focs_update_symbols)-len(finale_update_focs_g))))

Percentage of genes in GENCODE mapping: 0.973 (13,476 / 13,852)
Number of gene names that can be updated: 209
Number of gene names that cannot be updated: 164
Finale number of genes to update: 209 (0 could have an update, but it's not found in GENCODE.)


CPU times: user 26.2 s, sys: 104 ms, total: 26.3 s
Wall time: 26.3 s


In [369]:
%%time
# Now apply this dictionarry
new_targs_focs = red_df_focs.sym_id.replace(np.nan,'').apply(
                    lambda v: ';'.join([finale_update_focs_g.get(g,'') if g in focs_missing.values else g
                                        for g in v.split(',')]).strip(' ; '))
red_df_focs['new_targets'] = new_targs_focs

red_df_focs = red_df_focs.loc[~(red_df_focs['new_targets'].replace(np.nan,'')==''),:]

CPU times: user 1.46 s, sys: 36 ms, total: 1.5 s
Wall time: 1.5 s


Below is the part where I check if there's regions that overlap in the bed file. I merge them if so.

In [370]:
%%time
# Do not overlap book-ended regions
merged_df_focs = pbt.BedTool.from_dataframe(red_df_focs.iloc[:,[0,1,2]]).sort().merge(d=-1).to_dataframe()
print("Shape before merge: {:,} ; shape after merge: {:,}\n\n".format(red_df_focs.shape[0], merged_df_focs.shape[0]))

Shape before merge: 41,708 ; shape after merge: 34,333


CPU times: user 160 ms, sys: 268 ms, total: 428 ms
Wall time: 889 ms


This means that I need to perform a merging of regions...

Since there's no particular score associated to the enhancer/target predictions, I can simply assign the "new targets" names as the "name" of each regions ; then I perform the merging with bedops, and new regions will be named as the accumulation of the target names.

In [371]:
output_df_focs_tmp = red_df_focs.assign(name = red_df_focs.new_targets
                                 ).loc[:,['e_chr','e_start','e_end','name']]

output_df_focs_tmp.to_csv("/localtmp/moyon/tmp_focs_roadmap.tsv",header=False,index=False,sep="\t")

res_call = subprocess.call((command_line_merge
                           ).format(inputfile="/localtmp/moyon/tmp_focs_roadmap.tsv",
                                    outputfile="/localtmp/moyon/tmp_MERGED_focs_roadmap.tsv",
                                    delim="\|"
                                   ), shell=True)
print(res_call)

assert res_call==0, "Error while generating the merged version of the file."

finale_merged_df_focs = pd.read_table("/localtmp/moyon/tmp_MERGED_focs_roadmap.tsv",header=None,names=['chrom','start','end','name'])

display(finale_merged_df_focs.head())

0


,chrom,start,end,name
0,chr1,752447,752795,AL669831.1;AL669831.5
1,chr1,753344,753691,AL669831.1;AL669831.5
2,chr1,773729,773958,AL669831.1;AL669831.5
3,chr1,778091,778515,LINC00115;LINC01128
4,chr1,779960,780218,SAMD11|AL669831.1;AL669831.5


Here we sort-of have the possibility to compute a score : the number of times a gene appears in the list corresponds to the number of overlapping enhancers that were seen as associated to that gene.

In [372]:
%%time
merged_df_focs_names = finale_merged_df_focs['name'].apply(lambda v: from_list_genes_to_kv_pairs(v))
display(print(merged_df_focs_names.iloc[:5]))
print("\n")

0           score:1,targets:AL669831.1;AL669831.5
1           score:1,targets:AL669831.1;AL669831.5
2           score:1,targets:AL669831.1;AL669831.5
3             score:1,targets:LINC00115;LINC01128
4    score:1,targets:SAMD11;AL669831.1;AL669831.5
Name: name, dtype: object


None



CPU times: user 3min 5s, sys: 936 ms, total: 3min 6s
Wall time: 3min 6s


In [373]:
output_df_focs = finale_merged_df_focs.assign(name=merged_df_focs_names.values
                                 ).loc[:,['chrom','start','end','name']]

display(output_df_focs.head(3))

pbt.BedTool.from_dataframe(output_df_focs
                           ).sort().to_dataframe().to_csv(
    "/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/roadmap_enh.bed",
                     header=False,index=False,sep="\t")


if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/roadmap_enh.bed.gz"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/roadmap_enh.bed.gz")
    
if os.path.exists("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/roadmap_enh.bed.gz.tbi"):
    os.remove("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/roadmap_enh.bed.gz.tbi")
    

res_call = subprocess.call('bgzip {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/roadmap_enh.bed"), shell=True)
print(res_call)
# Now call tabix
res_call = subprocess.call('tabix -p bed -0 {}'.format("/users/ldog/moyon/Thesis/RegulationData/hg19/enhancers/FOCS/roadmap_enh.bed.gz"), shell=True)
print(res_call)
# Don't forget to run bgzip and tabix -0 -p bed

,chrom,start,end,name
0,chr1,752447,752795,"score:1,targets:AL669831.1;AL669831.5"
1,chr1,753344,753691,"score:1,targets:AL669831.1;AL669831.5"
2,chr1,773729,773958,"score:1,targets:AL669831.1;AL669831.5"


0
0
